In [2]:
import pandas as pd

In [3]:
import dvc.api

In [ ]:
# python main_inference_multimodal.py \
#         --model=LLM_Inference_Multimodal \
#         --model.llm_type="seqclf" \
#         --model.ckpt_path="models/multitask_multimodal_multilingual/version_14/epoch=3-step=217940.ckpt" \
#         --model.config_path="models/multitask_multimodal_multilingual/version_14/config.yaml" \
#         --model.task="seqclf_singlemodal_wishproducttitle2tax" \
#         --model.output_dir="models/multitask_multimodal_multilingual/version_14" \
#         --model.write_interval="batch" \
#         --data=JSONListData \
#         --data.llm_type="seqclf" \
#         --data.label_map_file="datasets/taxonomy/wish_v1.2.1_newtax_allpaths_withunknown.txt" \
#         --data.label_type="multilabel_taxonomy" \
#         --data.data_source_yaml_path="datasets/multimodal_multitask/liang_product_offshorev2_multilabel_seqclf.yaml" \
#         --data.input_dict="{'template': '{title}', 'task_prefix': 'Classify product title: '}" \
#         --data.output_dict="{'template': '{category_path}'}" \
#         --data.data_source_type="dvc" \
#         --data.model_name="microsoft/Multilingual-MiniLM-L12-H384" \
#         --data.batch_size=50 \
#         --data.max_length=50 \
#         --data.num_workers=0 \
#         --data.overwrite_cache=true \
#         --data.force_download_hfdata=true \
#         --trainer.logger=false \
#         --trainer.enable_checkpointing=false \
#         --trainer.accelerator gpu \
#         --trainer.strategy ddp

In [14]:
df_preds = pd.read_json('/workspaces/multitask-llm-rnd/modelling/models/multitask_multimodal_multilingual/version_14/seqclf-epoch=3-step=217940--liang_product_offshorev2_multilabel_seqclf--test.json', 
                        lines=True)

In [5]:
df_test = pd.read_json(dvc.api.get_url( 
    'datasets/data/liang_product/liang_product_classification_offshorev2only_test.json',
    repo='git@github.com:ContextLogic/multitask-llm-rnd.git'
), lines=True)

In [15]:
df_preds = df_preds.sort_values('batch_indices')

In [7]:
df_tax = pd.read_json(dvc.api.get_url(
    'datasets/data/taxonomy/wish_newtax.json',
    repo='git@github.com:ContextLogic/multitask-llm-rnd.git'
), lines=True)

In [8]:
df_tax_leaf = df_tax[df_tax.is_leaf]

In [9]:
valid_leaf_paths = set(df_tax_leaf['category_path'].str.lower())

In [16]:
df_preds['preds'].apply(lambda x: x[0][0] in valid_leaf_paths).all()

False

In [11]:
def get_top_leaf_pred(x):
    for i in x:
        if i[0] in valid_leaf_paths:
            return i[0]
    return 'unknown'

In [17]:
df_preds['top_pred'] = df_preds['preds'].apply(get_top_leaf_pred)

In [18]:
df_preds

,preds,batch_idx,dataloader_idx,batch_indices,logits,top_pred
80,"[[men's clothing, 0.685579061508178], [men's c...",0,0,0,"[-4.28412389755249, -7.331847190856934, -13.96...",men's clothing > sweaters > pullovers
0,"[[apparel accessories, 0.9797204136848451], [a...",0,0,1,"[3.877649068832397, -0.09184456616640001, -9.5...",apparel accessories > boys' accessories > boys...
40,[[electronic components & supplies > passive c...,0,0,2,"[-10.581339836120605, -12.941648483276367, -17...",electronic components & supplies > passive com...
78,"[[shoes > women's shoes, 0.9811940789222711], ...",0,0,3,"[-7.496616840362549, -12.602643013000488, -17....",shoes > women's shoes > boat shoes
62,"[[home & garden, 0.9522189497947691], [home & ...",0,0,4,"[-10.666418075561523, -11.249141693115234, -18...",home & garden > festive & party supplies > eve...
...,...,...,...,...,...,...
24629,"[[sports > fishing > fishing lines, 0.99918073...",70,0,24805,"[-10.627558708190918, -14.028524398803711, -20...",sports > fishing > fishing lines
24763,"[[automobiles & motorcycles, 0.996129870414733...",70,0,24806,"[-11.717824935913086, -14.009489059448242, -17...",automobiles & motorcycles > motorcycle accesso...
24668,"[[women's clothing, 0.9702994227409361], [wome...",70,0,24807,"[-6.61120080947876, -7.46343994140625, -16.468...",women's clothing > tops > blouses & shirts
24809,"[[watches, 0.9915645122528071], [watches > men...",70,0,24808,"[-9.42361068725586, -12.790079116821289, -14.4...",watches > men's watches > quartz watches


In [19]:
df_test.category_path.str.lower().apply(lambda x: x in valid_leaf_paths).all()

True

In [20]:
from eval_from_file import perf_eval_util


In [25]:
df_test.sample(1).to_dict('records')

[{'title': '11 Colors Fantasy Colored Nail Art Irregular Candy Paper Nail Art Aurora Fragment Nail Sequin Shiny Shell Paper Sequin 5g/bag',
  'description': 'It is recommended to buy multiple colors and use them together, the effect is more fashionable  Description: Material: Candy Paper Net content: 5g Nail product classification: Nail sequins Specification type: 5g irregular candy paper  Pagkage included: 1Bag x Candy Paper',
  'category_path': 'Beauty & Health > Nails Art & Tools > Rhinestones & Decorations',
  'category_id': 1078,
  'pid': '60b64662b172f765b048bf3d',
  'data_type': 'test_v2_offshore',
  'category': ['beauty & health',
   'nails art & tools',
   'rhinestones & decorations'],
  'category_pred': ['home & garden',
   'arts, crafts & sewing',
   'needle arts & crafts',
   'diamond painting cross-stitch']}]

In [21]:
df_test['category'] = df_test['category_path'].str.lower().apply(lambda x: x.split(' > '))
df_test['category_pred'] = df_preds['top_pred'].str.lower().apply(lambda x: x.split(' > '))

In [23]:
perf_eval_util(df_test, 1, 'category_pred')

,id,precision,recall,f1-score,support,level
0,apparel accessories,0.026087,0.022528,0.024177,799.000000,1
1,automobiles & motorcycles,0.087784,0.087093,0.087437,1906.000000,1
2,beauty & health,0.057234,0.053933,0.055534,1335.000000,1
3,cellphones & telecommunications,0.038141,0.038278,0.038209,836.000000,1
4,computer & office,0.017921,0.017986,0.017953,278.000000,1
5,consumer electronics,0.029268,0.030769,0.030000,585.000000,1
6,education & office supplies,0.010782,0.010929,0.010855,366.000000,1
7,electronic components & supplies,0.000000,0.000000,0.000000,28.000000,1
8,entertainment,0.015625,0.017964,0.016713,167.000000,1
9,food,0.000000,0.000000,0.000000,17.000000,1
